In [26]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
# citi training data
bike_data = pd.read_csv('data/201706-citibike-tripdata.csv.zip')

In [28]:
bike_data.shape

(1731594, 15)

In [29]:
bike_data.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [30]:
bike_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1397,2017-06-01 00:00:02,2017-06-01 00:23:19,515,W 43 St & 10 Ave,40.760094,-73.994618,3285,W 87 St & Amsterdam Ave,40.788390,-73.974700,26642,Subscriber,1967.0,1
1,1103,2017-06-01 00:00:13,2017-06-01 00:18:37,488,W 39 St & 9 Ave,40.756458,-73.993722,297,E 15 St & 3 Ave,40.734232,-73.986923,25656,Subscriber,1981.0,1
2,1810,2017-06-01 00:00:20,2017-06-01 00:30:31,461,E 20 St & 2 Ave,40.735877,-73.982050,465,Broadway & W 41 St,40.755136,-73.986580,21023,Subscriber,1982.0,1
3,1760,2017-06-01 00:00:24,2017-06-01 00:29:45,2009,Catherine St & Monroe St,40.711174,-73.996826,527,E 33 St & 2 Ave,40.744023,-73.976056,25718,Subscriber,1973.0,2
4,2165,2017-06-01 00:00:33,2017-06-01 00:36:38,360,William St & Pine St,40.707179,-74.008873,474,5 Ave & E 29 St,40.745168,-73.986831,18691,Subscriber,1985.0,1


In [46]:
bike_data['start station id'].value_counts()[:10]

519     16364
426     13542
497     11703
402     11388
514     11301
435     10889
459     10574
3255    10497
490     10009
368      9142
Name: start station id, dtype: int64

In [47]:
bike_data['end station id'].value_counts()[:10]

519     16089
426     14122
402     12006
497     11991
514     11272
459     11092
435     10993
3255    10645
368      9314
490      9213
Name: end station id, dtype: int64

In [51]:
bike_data['route_id'] = bike_data['start station id'].astype(str) + '_' + bike_data['end station id'].astype(str)

In [52]:
bike_data['route_id'].value_counts()[:10]

2006_2006    793
3182_3182    765
432_3263     739
2006_3282    723
3254_3182    678
3182_3254    672
514_426      651
426_514      648
281_281      609
3254_3254    604
Name: route_id, dtype: int64

In [55]:
bike_data.groupby('route_id').agg({'tripduration': np.mean})[:10]

,tripduration
route_id,
116_116,1135.256757
116_127,568.290323
116_128,851.000000
116_146,982.461538
116_150,1211.000000
116_151,1108.368421
116_152,1164.357143
116_153,913.875000
116_161,638.272727


## Plot street network and routes as interactive web maps

In [31]:
import osmnx as ox, networkx as nx
from IPython.display import IFrame

ox.config(log_console=True, use_cache=True)

In [32]:
import warnings
import requests
import contextlib

try:
    from functools import partialmethod
except ImportError:
    # Python 2 fallback: https://gist.github.com/carymrobbins/8940382
    from functools import partial

    class partialmethod(partial):
        def __get__(self, instance, owner):
            if instance is None:
                return self

            return partial(self.func, instance, *(self.args or ()), **(self.keywords or {}))

@contextlib.contextmanager
def no_ssl_verification():
    old_request = requests.Session.request
    requests.Session.request = partialmethod(old_request, verify=False)

    warnings.filterwarnings('ignore', 'Unverified HTTPS request')
    yield
    warnings.resetwarnings()

    requests.Session.request = old_request

In [33]:
# Download the street network for Peidmont, CA
with no_ssl_verification():
    G = ox.graph_from_place('Piedmont, California, USA', network_type='drive')    

/home/s4szqx/anaconda3/lib/python3.6/site-packages/osmnx/core.py:111: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/7a10ef7f51a32868ea1208e6d6162cd1.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))
/home/s4szqx/anaconda3/lib/python3.6/site-packages/osmnx/core.py:111: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/d129ff1e8f7404f85891e99f8d59cd81.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))


In [41]:
graph_map = ox.plot_graph_folium(G, popup_attribute='name', edge_width=2)

In [35]:
import folium

m = folium.Map(location=[45.5236, -122.6750])

In [36]:
m

In [37]:
map_5 = folium.Map(location=[45.5236, -122.6750],
                   zoom_start=13)

folium.RegularPolygonMarker(
    [45.5012, -122.6655],
    popup='Ross Island Bridge',
    fill_color='#132b5e',
    number_of_sides=3,
    radius=10
    ).add_to(map_5)
folium.RegularPolygonMarker(
    [45.5132, -122.6708],
    popup='Hawthorne Bridge',
    fill_color='#45647d',
    number_of_sides=4,
    radius=10
    ).add_to(map_5)
folium.RegularPolygonMarker(
    [45.5275, -122.6692],
    popup='Steel Bridge',
    fill_color='#769d96',
    number_of_sides=6,
    radius=10
    ).add_to(map_5)
folium.RegularPolygonMarker(
    [45.5318, -122.6745],
    popup='Broadway Bridge',
    fill_color='#769d96',
    number_of_sides=8,
    radius=10
    ).add_to(map_5)
map_5

In [38]:
# use networkx to calculate the shortest path between two nodes
origin_node = list(G.nodes())[0]
destination_node = list(G.nodes())[-1]
route = nx.shortest_path(G, origin_node, destination_node)

In [39]:
# plot the route with folium
route_map = ox.plot_route_folium(G, route)

In [40]:
route_map

In [43]:
# plot the route with folium on top of the previously created graph_map
route_graph_map = ox.plot_route_folium(G, route, popup_attribute='length')

route_graph_map